In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sms
import statsmodels.api as sm
%matplotlib inline
import seaborn as sns
from pandas.plotting import lag_plot
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf


In [ ]:
df = pd.read_excel('Airlines+Data.xlsx',index_col=0, parse_dates=['Month'])


In [ ]:
df.head()

#Exploartory data analysis

In [ ]:
df.isna().sum()

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(10,7))
plt.plot(df)

In [ ]:
df.hist()

In [ ]:
#Autocorrelation Plot
from statsmodels.graphics.tsaplots import plot_acf
plt.figure(figsize=(20,5))
plot_acf(df,lags=95)
plt.show()

In [ ]:
#PACF
from statsmodels.graphics.tsaplots import plot_pacf
plt.figure(figsize=(20,5))
plot_pacf(df.Passengers,lags=45)
plt.show()

In [ ]:
#Lag Plot
plt.figure(figsize=(13,5))
pd.plotting.lag_plot(df)

#Upsampling


In [ ]:
#Upsampling
upsampled=df.resample('D').mean()
upsampled.head()

In [ ]:
interpolated=upsampled.interpolate(method='linear')
interpolated.head(10)

In [ ]:
interpolated.plot()

#Downsampling


In [ ]:

downsampled=df.resample('Q').mean()
downsampled.head()

In [ ]:
plt.plot(downsampled)

In [ ]:
df.columns = df.columns.str.strip()
df = df.reset_index()


In [ ]:
df["month"]=df['Month'].dt.strftime("%b")
df["year"]=df['Month'].dt.strftime("%Y")
df['log_passengers']=np.log(df['Passengers'])
df['t']=np.arange(1,97)
df['t_square']=np.square(df['t'])
df1=pd.get_dummies(df['month'])
df=pd.concat([df,df1],axis=1)
df.head()

In [ ]:
#Heatmap
hm=pd.pivot_table(data=df,values="Passengers",columns="month",index='year')
plt.figure(figsize=(15,7))
sns.heatmap(hm,annot=True,fmt='g')

In [ ]:
#Multiplicative Seasonality Decomposition

decompose_ts_add = seasonal_decompose(df.Passengers,period=12, model='multiplicative')
with plt.rc_context():
    plt.rc("figure", figsize=(14,10))
    decompose_ts_add.plot()
    plt.show()

**Model building**


In [ ]:
#Data Spliting
Train = df.head(84)
Test = df.tail(12)

In [ ]:
#Linear Model
import statsmodels.formula.api as smf

linear_model = smf.ols('Passengers~t',data=Train).fit()
pred_linear =  pd.Series(linear_model.predict(pd.DataFrame(Test['t'])))
rmse_linear = np.sqrt(np.mean((np.array(Test['Passengers'])-np.array(pred_linear))**2))
rmse_linear

In [ ]:
#Exponential
Exp = smf.ols('log_passengers~t',data=Train).fit()
pred_Exp = pd.Series(Exp.predict(pd.DataFrame(Test['t'])))
rmse_Exp = np.sqrt(np.mean((np.array(Test['Passengers'])-np.array(np.exp(pred_Exp)))**2))
rmse_Exp

In [ ]:
#Quadratic
Quad = smf.ols('Passengers~t+t_square',data=Train).fit()
pred_Quad = pd.Series(Quad.predict(Test[["t","t_square"]]))
rmse_Quad = np.sqrt(np.mean((np.array(Test['Passengers'])-np.array(pred_Quad))**2))
rmse_Quad

In [ ]:
#Additive seasonality
add_sea = smf.ols('Passengers~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=Train).fit()
pred_add_sea = pd.Series(add_sea.predict(Test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov']]))
rmse_add_sea = np.sqrt(np.mean((np.array(Test['Passengers'])-np.array(pred_add_sea))**2))
rmse_add_sea

In [ ]:
#Additive Seasonality Quadratic
add_sea_Quad = smf.ols('Passengers~t+t_square+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=Train).fit()
pred_add_sea_quad = pd.Series(add_sea_Quad.predict(Test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','t','t_square']]))
rmse_add_sea_quad = np.sqrt(np.mean((np.array(Test['Passengers'])-np.array(pred_add_sea_quad))**2))
rmse_add_sea_quad

In [ ]:
##Multiplicative Seasonality
Mul_sea = smf.ols('log_passengers~Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = Train).fit()
pred_Mult_sea = pd.Series(Mul_sea.predict(Test))
rmse_Mult_sea = np.sqrt(np.mean((np.array(Test['Passengers'])-np.array(np.exp(pred_Mult_sea)))**2))
rmse_Mult_sea

In [ ]:
#Multiplicative Additive Seasonality
Mul_Add_sea = smf.ols('log_passengers~t+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data = Train).fit()
pred_Mult_add_sea = pd.Series(Mul_Add_sea.predict(Test))
rmse_Mult_add_sea = np.sqrt(np.mean((np.array(Test['Passengers'])-np.array(np.exp(pred_Mult_add_sea)))**2))
rmse_Mult_add_sea

In [ ]:
#Multiplicative Seasonality Quadratic
mult_sea_Quad = smf.ols('log_passengers~t+t_square+Jan+Feb+Mar+Apr+May+Jun+Jul+Aug+Sep+Oct+Nov',data=Train).fit()
pred_mult_sea_quad = pd.Series(mult_sea_Quad.predict(Test[['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','t','t_square']]))
rmse_mult_sea_quad = np.sqrt(np.mean((np.array(Test['Passengers'])-np.array(pred_mult_sea_quad))**2))
rmse_mult_sea_quad

In [ ]:
data = {"MODEL":pd.Series(["rmse_linear","rmse_Exp","rmse_Quad","rmse_add_sea","rmse_add_sea_quad","rmse_Mult_sea","rmse_Mult_add_sea"]),"RMSE_Values":pd.Series([rmse_linear,rmse_Exp,rmse_Quad,rmse_add_sea,rmse_add_sea_quad,rmse_Mult_sea,rmse_Mult_add_sea])}
table_rmse=pd.DataFrame(data)
table_rmse.sort_values(['RMSE_Values'])

In [ ]:
""" Dickey Fuller Test for testing the data is stationary or not """

def adf_test(series):
    result = adfuller(series)
    print('ADF Statistics: {}'.format(result[0]))
    print('p-value: {}'.format(result[1]))
    if result[1] <= 0.05:
       print("Strong evidence against the null hypothesis,reject the null hypothesis.Data has no unit root and is stationary")
    else:
       print("Weak evidence against null hypothesis,time series has unit root , indicating it is non -statinary")


adf_test(df['Passengers'])



In [ ]:

""" using techniques Differencing   """

df['Passengers First Difference']=df['Passengers']-df['Passengers'].shift(1)
df.head()

adf_test(df['Passengers First Difference'].dropna())
# data is still not stationary.


In [ ]:

""" using second differencing technique """

df['Passengers Second Difference'] = df['Passengers First Difference']-df['Passengers First Difference'].shift(1)

adf_test(df['Passengers Second Difference'].dropna())



In [ ]:

""" we are checking for over a year now  ( we are looking for seasonal differences now and sometimes ARIMA doesnot work well for seasonal data but SARIMAX Does )"""


In [ ]:
  df['Passengers 12 Difference']=df['Passengers']-df['Passengers'].shift(12)
adf_test(df['Passengers 12 Difference'].dropna())



In [ ]:
"""   Fit the ARIMA model  """
model = ARIMA(Train['Passengers'], order=(11, 2, 2))
model_fit = model.fit()

# Make predictions on the testing set
predictions = model_fit.forecast(steps=len(Test))

predictions.plot()


In [ ]:
#predictions

# Check the shape of the input arrays
if Test['Passengers'].shape[0] > 1 and predictions.shape[0] > 1:
    # Calculate the mean squared error
    mse = mean_squared_error(Test['Passengers'], predictions)

    # Calculate the root mean squared error
    rmse = np.sqrt(mse)
    print('RMSE:', rmse)
else:
    print('Error: Input arrays have only one element')



In [ ]:

model_fit.summary()
Test['predictions'] = predictions

Test[['Passengers','predictions']].plot()



In [ ]:
"""  SARIMAX   """
# Fit SARIMA model
model_SARIMA = SARIMAX(Train['Passengers'], order=(11, 2, 2), seasonal_order=(0, 0, 0, 12))
model_SARIMA_fit = model_SARIMA.fit()

# Make predictions on the testing set
predictions_SARIMA = model_SARIMA_fit.forecast(steps=len(Test))

# Check the shape of the input arrays
if Test['Passengers'].shape[0] > 1 and predictions_SARIMA.shape[0] > 1:
    # Calculate the mean squared error
    mse = mean_squared_error(Test['Passengers'], predictions_SARIMA)

In [ ]:
# Calculate the root mean squared error
rmse = np.sqrt(mse)
print('RMSE:', rmse)


In [ ]:
# Display model summary
model_SARIMA_fit.summary()

# Add predictions to test_data
Test['predictions_SARIMA'] = predictions_SARIMA
# Plot actual vs predicted values
Test[['Passengers', 'predictions_SARIMA']].plot()
plt.show()



In [ ]:
acf12 = plot_acf(df['Passengers 12 Difference'].dropna())
pacf12 = plot_pacf(df['Passengers 12 Difference'].dropna())


In [ ]:
""" considering the data to be seasonal checking with the results for arima and sarimax models """

from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt

# Fit ARIMA model
model_ARIMA = ARIMA(Train['Passengers'], order=(3, 0, 5))
model_ARIMA_fit = model_ARIMA.fit()
predictions_ARIMA = model_ARIMA_fit.forecast(steps=len(Test))

# Fit SARIMA model
model_SARIMA = SARIMAX(Train['Passengers'], order=(3, 0, 5), seasonal_order=(0, 1, 0, 12))
model_SARIMA_fit = model_SARIMA.fit()
predictions_SARIMA = model_SARIMA_fit.forecast(steps=len(Test))

# Calculate RMSE for ARIMA
mse_ARIMA = mean_squared_error(Test['Passengers'], predictions_ARIMA)
rmse_ARIMA = np.sqrt(mse_ARIMA)
print('RMSE (ARIMA):', rmse_ARIMA)

# Calculate RMSE for SARIMA
mse_SARIMA = mean_squared_error(Test['Passengers'], predictions_SARIMA)
rmse_SARIMA = np.sqrt(mse_SARIMA)
print('RMSE (SARIMA):', rmse_SARIMA)


In [ ]:

# Plot actual vs predicted values for both models
plt.figure(figsize=(10, 6))
plt.plot(Test['Passengers'], label='Actual')
plt.plot(predictions_ARIMA, label='ARIMA Predictions')
plt.plot(predictions_SARIMA, label='SARIMA Predictions')
plt.legend()
plt.title('Actual vs Predicted Passengers')
plt.xlabel('Month')
plt.ylabel('Passengers')
plt.show()


In [ ]:
#Inference:
#SARIMA gives better results

In [ ]:
#Mean Absolute error

def MAPE(pred,org):
  temp = np.abs((pred-org)/org)*100
  return np.mean(temp)

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing


ses_model = SimpleExpSmoothing(Train["Passengers"]).fit(smoothing_level=0.2)
pred_ses = ses_model.predict(start=Test.index[0], end=Test.index[-1])

MAPE(pred_ses, Test.Passengers)


In [ ]:
# performing holt method
from statsmodels.tsa.holtwinters import ExponentialSmoothing

holt_model = ExponentialSmoothing(Train["Passengers"], trend='add').fit()
pred_holt = holt_model.predict(start=Test.index[0], end=Test.index[-1])

MAPE(pred_holt, Test.Passengers)


In [ ]:
#perform holt winter exponentail smoothing with additive trend and additive seaonality


from statsmodels.tsa.holtwinters import ExponentialSmoothing

holt_winter_model = ExponentialSmoothing(Train["Passengers"], trend='add', seasonal='add', seasonal_periods=12).fit()
pred_holt_winter = holt_winter_model.predict(start=Test.index[0], end=Test.index[-1])

MAPE(pred_holt_winter, Test.Passengers)


In [ ]:
#performing holt winter exponentail smoothing with additive trend and multiplicative seaonality


from statsmodels.tsa.holtwinters import ExponentialSmoothing

holt_winter_model = ExponentialSmoothing(Train["Passengers"], trend='add', seasonal='mul', seasonal_periods=12).fit()
pred_holt_winter = holt_winter_model.predict(start=Test.index[0], end=Test.index[-1])

MAPE(pred_holt_winter, Test.Passengers)


In [ ]:
#final model by combining Train and test
hwe_model_add_add = ExponentialSmoothing(df.Passengers,seasonal='mul', seasonal_periods=12).fit()
#forecasting for next 10 time periods
hwe_model_add_add.forecast(10)